In [92]:
data_dir = 'data'
result_dir = 'result'

# Import

In [93]:
import pandas as pd
import numpy as np
# import datetime
import copy

import warnings
warnings.filterwarnings('ignore')


window_width = 7200 #10800 #120 #60 #600 # 120  # seconds
window_stride = 3600 #10800 #60 #60 #30 #600 # 60  # seconds

start_idx = 0
end_idx = 12000

af = 'u'
pf = f'{window_width}-{window_stride}.__2__'
ra = '__97.39__'

test_attack_csvname = 'dfo_new1'

# Aggregate flows

In [94]:
path = f'{result_dir}/{test_attack_csvname}.csv'
df = pd.read_csv(path, index_col=[0])

In [95]:
df['Conversation'] = df['SrcAddr']+' -> '+df['DstAddr']

## State

In [96]:
df['State_orig'] = df['State']
df.loc[(df['State_orig'].str.len() > 2) & (df['State_orig'].str.contains('_')), 'State'] = 'alltcp'

## Aggregate

In [97]:
df

,StreamID,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,Label,LabelStr,PktsPerSec,BytesPerSec,SrcBytesPerSec,BytesPerPkt,State_orig,Conversation
1975,1976,2022-07-25 23:30:50.093002,3571.229975,tcp,41.232.73.23,6669,<?>,150.35.87.168,1027,alltcp,5,flow=From-Botnet-V44-TCP-CC107-IRC-Not-Encrypted,0.022401,1.648452,1.050058,73.587500,alltcp,41.232.73.23 -> 150.35.87.168
1976,1977,2022-07-25 23:31:50.093002,3571.229975,tcp,41.232.73.23,6668,<?>,150.35.87.168,1027,alltcp,5,flow=From-Botnet-V44-TCP-CC107-IRC-Not-Encrypted,49.979010,14750.948178,14743.933560,88.585199,alltcp,41.232.73.23 -> 150.35.87.168
1978,1979,2022-07-25 23:33:50.093002,3571.229975,tcp,41.232.73.23,6668,<?>,150.35.87.168,1027,alltcp,5,flow=From-Botnet-V44-TCP-CC107-IRC-Not-Encrypted,49.979010,24906.267209,24906.267209,73.202824,alltcp,41.232.73.23 -> 150.35.87.168
51590,51591,2022-07-26 00:32:56.114638,3527.809143,tcp,41.232.73.23,6667,<?>,150.35.87.168,1027,alltcp,5,flow=From-Botnet-V44-TCP-CC107-IRC-Not-Encrypted,0.062078,5.494345,2.641583,88.506849,alltcp,41.232.73.23 -> 150.35.87.168


In [98]:
df['StartTimeInt'] = pd.to_datetime(df['StartTime']).astype(np.int64)*1e-9
datetime_start = df['StartTimeInt'].min()

df['Window_lower'] = (df['StartTimeInt'] - datetime_start - window_width) / window_stride + 1
df['Window_lower'].clip(lower=0, inplace=True)
df['Window_upper_excl'] = (df['StartTimeInt'] - datetime_start) / window_stride + 1
df = df.astype({'Window_lower': int, 'Window_upper_excl': int})
# df#.head()

In [99]:
def RU(df):
    if df.shape[0] == 1:
        return 1.0
    else:
        proba = df.value_counts()/df.shape[0]
        h = proba*np.log10(proba)
        return -h.sum()/np.log10(df.shape[0])

def top(df):
    return df.mode()[0]

def process_window(X, i):
    if len(df.loc[(df['Window_lower'] <= i) & (df['Window_upper_excl'] > i)]) == 0:
        return X

    print('-----------------------\nProcess window: ', i)

    """ All flows from this SrcAddr in this time window """
    gb_SrcAddr = df.loc[(df['Window_lower'] <= i) & (df['Window_upper_excl'] > i)].groupby(['Conversation','SrcAddr','DstAddr','State','Proto'])

    r = {
        'StreamID': ['unique'],

        'Sport': ['mean', 'max'],
        
        'PktsPerSec': ['mean'],
        'BytesPerSec': ['mean', 'max'],
        'SrcBytesPerSec': ['max'],
        'BytesPerPkt': ['mean', 'max'],
    }
    
    agg = gb_SrcAddr.agg(r)
    agg['Label'] = gb_SrcAddr['Label'].agg(lambda x: x.mode()[0])

    tmp = gb_SrcAddr.size().to_frame(name='n_flows').join(agg).reset_index().assign(window_id=i)
    X = X.append(tmp)
    del gb_SrcAddr

    return X

In [100]:
dfg = pd.DataFrame()
for i in range(start_idx, end_idx):
    dfg = process_window(dfg, i)

-----------------------
Process window:  0
-----------------------
Process window:  1


In [101]:
dfg.columns = ['_'.join(x) if isinstance(x, tuple) else x for x in dfg.columns.ravel()]
dfg = dfg.rename(columns={'Label_': 'Label', 'SrcAddr_': 'SrcAddr', 'StartTime_min': 'StartTime_earliest', 'StartTime_max': 'StartTime_latest'})
dfg = dfg.loc[dfg['n_flows'] > 1]
dfg

,Conversation,SrcAddr,DstAddr,State,Proto,n_flows,StreamID_unique,Sport_mean,Sport_max,PktsPerSec_mean,BytesPerSec_mean,BytesPerSec_max,SrcBytesPerSec_max,BytesPerPkt_mean,BytesPerPkt_max,Label,window_id
0,41.232.73.23 -> 150.35.87.168,41.232.73.23,150.35.87.168,alltcp,tcp,4,"[1976, 1977, 1979, 51591]",6668.0,6669,25.010625,9916.089546,24906.267209,24906.267209,80.970593,88.585199,5,0


In [102]:
del df

# Encode

In [103]:
""" load the encoder """
import pickle

ct_train = pickle.load(open(f'{data_dir}/{af}4-3.ohe.{pf}.pkl', 'rb'))
ms = pickle.load(open(f'{data_dir}/{af}4-3.mms.{pf}.pkl', 'rb'))

## One-hot endode

In [104]:
dfg.loc[(dfg['State'].str.contains('_')), 'State'] = 'alltcp'

In [105]:
""" replace infrequent state with other """
common_state_list = list(dfg['State'].value_counts()[:20].index)
print(common_state_list)
dfg.loc[(~dfg['State'].str.contains('_')) & (~dfg['State'].isin(common_state_list)), 'State'] = 'other'

""" replace infrequent proto with other """
common_proto_list = list(dfg['Proto'].value_counts()[:3].index)
print(common_proto_list)
dfg.loc[~dfg['Proto'].isin(common_proto_list), 'Proto'] = 'other'

['alltcp']
['tcp']


In [106]:
""" get categories to encode """
#? get these values from u4-3.enc when encoding train data.
common_proto_list = ['udp', 'tcp', 'icmp']
common_state_list = ['CON', 'alltcp', 'INT', 'ECR', 'REQ', 'RED', 'ECO', 'URP', 'URH', 'RSP', 'TXD', 'URHPRO', 'URN', 'URO', 'URFIL', 'DCE', 'URF', 'nanvalue']

proto_list = common_proto_list + ['other']
state_list = common_state_list + ['other']
categories = [proto_list, state_list]

onehot_cols_proto = ['P_'+p for p in proto_list]
onehot_cols_state = ['S_'+p for p in state_list]
onehot_cols = onehot_cols_proto + onehot_cols_state

print('proto_list', proto_list)
print('state_list', state_list)
print('categories', categories)
print('onehot_cols', onehot_cols)

Xstr_enc = ct_train.transform(dfg[['Proto','State']])
Xstr_enc = Xstr_enc.astype('float64').toarray()
df_Xstr_enc = pd.DataFrame(Xstr_enc, columns=onehot_cols)

proto_list ['udp', 'tcp', 'icmp', 'other']
state_list ['CON', 'alltcp', 'INT', 'ECR', 'REQ', 'RED', 'ECO', 'URP', 'URH', 'RSP', 'TXD', 'URHPRO', 'URN', 'URO', 'URFIL', 'DCE', 'URF', 'nanvalue', 'other']
categories [['udp', 'tcp', 'icmp', 'other'], ['CON', 'alltcp', 'INT', 'ECR', 'REQ', 'RED', 'ECO', 'URP', 'URH', 'RSP', 'TXD', 'URHPRO', 'URN', 'URO', 'URFIL', 'DCE', 'URF', 'nanvalue', 'other']]
onehot_cols ['P_udp', 'P_tcp', 'P_icmp', 'P_other', 'S_CON', 'S_alltcp', 'S_INT', 'S_ECR', 'S_REQ', 'S_RED', 'S_ECO', 'S_URP', 'S_URH', 'S_RSP', 'S_TXD', 'S_URHPRO', 'S_URN', 'S_URO', 'S_URFIL', 'S_DCE', 'S_URF', 'S_nanvalue', 'S_other']


In [107]:
dfg = pd.concat([dfg.reset_index(), df_Xstr_enc], axis=1).drop(columns=['index'])
dfg.head()

,Conversation,SrcAddr,DstAddr,State,Proto,n_flows,StreamID_unique,Sport_mean,Sport_max,PktsPerSec_mean,...,S_RSP,S_TXD,S_URHPRO,S_URN,S_URO,S_URFIL,S_DCE,S_URF,S_nanvalue,S_other
0,41.232.73.23 -> 150.35.87.168,41.232.73.23,150.35.87.168,alltcp,tcp,4,"[1976, 1977, 1979, 51591]",6668.0,6669,25.010625,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Numeric

In [108]:
numeric_cols = [
       'n_flows',
       'Sport_mean','Sport_max',
       'BytesPerPkt_mean','PktsPerSec_mean','BytesPerSec_mean','BytesPerPkt_max','BytesPerSec_max','SrcBytesPerSec_max',
]
dfg[numeric_cols] = ms.transform(dfg[numeric_cols].values)

## X y

In [109]:
fts_cols = numeric_cols + onehot_cols
X, y = dfg[fts_cols].values, dfg['Label'].values

y[y != 5] = 0
y[y == 5] = 1

print(X.shape, y.shape)

(1, 32) (1,)


In [110]:
X

array([[4.65213649e-05, 1.01746382e-01, 1.01759312e-01, 5.28652307e-02,
        5.00845904e-05, 5.00180146e-05, 5.65317160e-02, 8.44540544e-05,
        8.44778980e-05, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [111]:
del dfg

# Detect

In [112]:
""" load the model """
import pickle

model = pickle.load(open(f'{result_dir}/{af}4-5.{pf}.model_lr.{ra}.pkl', 'rb'))

In [113]:
""" choose what features to feed the model """
selected_fts = [
    'BytesPerPkt_mean',
    'PktsPerSec_mean','BytesPerSec_mean',
    'Sport_max','Sport_mean',
    'n_flows',
    'BytesPerPkt_max','BytesPerSec_max','SrcBytesPerSec_max',
    'P_tcp','P_udp','P_other',

    'S_CON', 'S_alltcp', 'S_INT', 'S_RED', 'S_other', 'S_ECO', 
]

#? index of top features
ft_idx = [fts_cols.index(ft) for ft in selected_fts]
print(ft_idx)

X = X[:,ft_idx]
print(X.shape)

[3, 4, 5, 2, 1, 0, 6, 7, 8, 10, 9, 12, 13, 14, 15, 18, 31, 19]
(1, 18)


In [114]:
""" check output of the model """
y_pred = model.predict(X)
print('y true', y) 
print('y pred', y_pred)

y true [1]
y pred [0]


The model is exploited successfully.

In [115]:
#? save ?
np.save(f'{result_dir}/{test_attack_csvname}.X.npy', X)
np.save(f'{result_dir}/{test_attack_csvname}.y.npy', y)